### 23.06.21
(mission 23.06.19 MON)


Hi I'm novice in data analysis and I'm studying for certification about data analyzing, in Korea.


### data and libraries

In [1]:
import pandas as pd
import numpy as np
train = pd.read_csv('/kaggle/input/playground-series-s3e17/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s3e17/test.csv')
submission_sample = pd.read_csv('/kaggle/input/playground-series-s3e17/sample_submission.csv')

from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler

from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, StackingClassifier

from sklearn.metrics import roc_auc_score

## data check and simple EDA

In [2]:
train.head(3)

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,0,L50096,L,300.6,309.6,1596,36.1,140,0,0,0,0,0,0
1,1,M20343,M,302.6,312.1,1759,29.1,200,0,0,0,0,0,0
2,2,L49454,L,299.3,308.5,1805,26.5,25,0,0,0,0,0,0


In [3]:
print(train.shape, test.shape, submission_sample.shape)

(136429, 14) (90954, 13) (90954, 2)


In [4]:
print(train.info(), '\n')
print(test.info())

# don't have to missing values check

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136429 entries, 0 to 136428
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       136429 non-null  int64  
 1   Product ID               136429 non-null  object 
 2   Type                     136429 non-null  object 
 3   Air temperature [K]      136429 non-null  float64
 4   Process temperature [K]  136429 non-null  float64
 5   Rotational speed [rpm]   136429 non-null  int64  
 6   Torque [Nm]              136429 non-null  float64
 7   Tool wear [min]          136429 non-null  int64  
 8   Machine failure          136429 non-null  int64  
 9   TWF                      136429 non-null  int64  
 10  HDF                      136429 non-null  int64  
 11  PWF                      136429 non-null  int64  
 12  OSF                      136429 non-null  int64  
 13  RNF                      136429 non-null  int64  
dtypes: f

In [5]:
# object types : 'Product ID' -> drop
print(train['Product ID'].value_counts(), '\n')
print(test['Product ID'].value_counts())

L53257    139
L49056    131
L53271    131
L54275    120
L48892    120
         ... 
M24614      1
H39399      1
M15853      1
H32637      1
L48036      1
Name: Product ID, Length: 9976, dtype: int64 

L56257    98
L48892    91
L53258    86
L53257    85
L53271    83
          ..
L47342     1
L50295     1
M19622     1
L47273     1
H36725     1
Name: Product ID, Length: 9909, dtype: int64


In [6]:
# object types : 'Type' (prodict id's first string) -> encoding
print(train['Type'].value_counts(), '\n')
print(test['Type'].value_counts())

L    95354
M    32152
H     8923
Name: Type, dtype: int64 

L    63438
M    21535
H     5981
Name: Type, dtype: int64


In [7]:
# int types 
# print(train.select_dtypes('int64').columns)

int64types_list = list(train.select_dtypes('int64').columns)
for i in int64types_list:
    print('### {}'.format(i))
    print(train[i].value_counts(), '\n')

# 'Machine failure' : target
# 'TWF', 'HDF', 'PWF', 'OSF', 'RNF' : binary features

### id
0         1
90948     1
90962     1
90961     1
90960     1
         ..
45471     1
45470     1
45469     1
45468     1
136428    1
Name: id, Length: 136429, dtype: int64 

### Rotational speed [rpm]
1452    1119
1447     937
1435     931
1418     894
1469     868
        ... 
2006       1
2617       1
2676       1
2760       1
2169       1
Name: Rotational speed [rpm], Length: 952, dtype: int64 

### Tool wear [min]
0      3790
2      1771
119    1448
59     1427
5      1419
       ... 
241       7
237       6
239       6
253       3
251       1
Name: Tool wear [min], Length: 246, dtype: int64 

### Machine failure
0    134281
1      2148
Name: Machine failure, dtype: int64 

### TWF
0    136217
1       212
Name: TWF, dtype: int64 

### HDF
0    135725
1       704
Name: HDF, dtype: int64 

### PWF
0    136102
1       327
Name: PWF, dtype: int64 

### OSF
0    135889
1       540
Name: OSF, dtype: int64 

### RNF
0    136121
1       308
Name: RNF, dtype: int64 



In [8]:
train.describe()
# it has ideal data except 'Torque [Nm]'s min value. (3.800000)

,id,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
count,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000
mean,68214.000000,299.862776,309.941070,1520.331110,40.348643,104.408901,0.015744,0.001554,0.005160,0.002397,0.003958,0.002258
std,39383.804275,1.862247,1.385173,138.736632,8.502229,63.965040,0.124486,0.039389,0.071649,0.048899,0.062789,0.047461
min,0.000000,295.300000,305.800000,1181.000000,3.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,34107.000000,298.300000,308.700000,1432.000000,34.600000,48.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,68214.000000,300.000000,310.000000,1493.000000,40.400000,106.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,102321.000000,301.200000,310.900000,1580.000000,46.100000,159.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,136428.000000,304.400000,313.800000,2886.000000,76.600000,253.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
test.describe()
# but the '3.800000' is exist in test data, too. I'll treat it as normal data.

,id,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],TWF,HDF,PWF,OSF,RNF
count,90954.000000,90954.000000,90954.000000,90954.000000,90954.000000,90954.000000,90954.000000,90954.000000,90954.000000,90954.00000,90954.000000
mean,181905.500000,299.859493,309.939375,1520.528179,40.335191,104.293962,0.001473,0.005343,0.002353,0.00387,0.002309
std,26256.302529,1.857562,1.385296,139.970419,8.504683,63.871092,0.038355,0.072903,0.048449,0.06209,0.047995
min,136429.000000,295.300000,305.700000,1168.000000,3.800000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,159167.250000,298.300000,308.700000,1432.000000,34.600000,48.000000,0.000000,0.000000,0.000000,0.00000,0.000000
50%,181905.500000,300.000000,310.000000,1493.000000,40.500000,106.000000,0.000000,0.000000,0.000000,0.00000,0.000000
75%,204643.750000,301.200000,310.900000,1579.000000,46.200000,158.000000,0.000000,0.000000,0.000000,0.00000,0.000000
max,227382.000000,304.400000,313.800000,2886.000000,76.600000,253.000000,1.000000,1.000000,1.000000,1.00000,1.000000


In [10]:
train.select_dtypes(exclude = 'O').corr()[['Machine failure']]

,Machine failure
id,0.001500
Air temperature [K],0.067744
Process temperature [K],0.030966
Rotational speed [rpm],-0.060868
Torque [Nm],0.145793
Tool wear [min],0.056448
Machine failure,1.000000
TWF,0.311919
HDF,0.564507
PWF,0.383941


## Preprocessing

In [11]:
y_train = train.pop('Machine failure')

df = pd.concat([train, test], axis = 0)
print(df.shape, train.shape, test.shape)

(227383, 13) (136429, 13) (90954, 13)


In [12]:
# 1. Object type - drop the column 'Product ID'
df = df.drop(columns = 'Product ID')
print(df.shape, train.shape, test.shape)

(227383, 12) (136429, 13) (90954, 13)


In [13]:
# 2. Object type - encoding the column 'Type'
# one-hot encoding with pandas : because it has little unbalanced categories
df = pd.get_dummies(data = df, columns = ['Type'], drop_first = True)

In [14]:
print(df.select_dtypes(exclude = 'O').columns)

Index(['id', 'Air temperature [K]', 'Process temperature [K]',
       'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]', 'TWF',
       'HDF', 'PWF', 'OSF', 'RNF', 'Type_L', 'Type_M'],
      dtype='object')


In [15]:
# 3. Numeric type - 'Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]'
# : StandardScaling 

X_train, X_test = df[:train.shape[0]], df[train.shape[0]:]

scaler = StandardScaler()
# scaler.fit(X_train[['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]']])
X_train.loc[: ,'Air temperature [K]':'Rotational speed [rpm]'] = scaler.fit_transform(X_train.loc[: ,'Air temperature [K]':'Rotational speed [rpm]'])
X_test.loc[:, 'Air temperature [K]':'Rotational speed [rpm]'] =  scaler.transform(X_test.loc[: ,'Air temperature [K]':'Rotational speed [rpm]'])
X_train

/tmp/ipykernel_20/2714640218.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.loc[: ,'Air temperature [K]':'Rotational speed [rpm]'] = scaler.fit_transform(X_train.loc[: ,'Air temperature [K]':'Rotational speed [rpm]'])
/tmp/ipykernel_20/2714640218.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.loc[:, 'Air temperature [K]':'Rotational speed [rpm]'] =  scaler.transform(X_test.loc[: ,'Air temperature [K]':'Rotational speed [rpm]'])


,id,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],TWF,HDF,PWF,OSF,RNF,Type_L,Type_M
0,0,0.395880,-0.246230,0.545416,36.1,140,0,0,0,0,0,1,0
1,1,1.469856,1.558605,1.720308,29.1,200,0,0,0,0,0,0,1
2,2,-0.302204,-1.040358,2.051873,26.5,25,0,0,0,0,0,1,0
3,3,0.610675,0.692284,0.026445,44.3,197,0,0,0,0,0,1,0
4,4,-1.000288,-0.679391,0.869773,35.4,34,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
136424,136424,0.127386,1.053251,0.069693,37.5,210,0,0,0,0,0,0,1
136425,136425,-1.268781,-1.040358,-0.528565,49.1,2,0,0,0,0,0,0,0
136426,136426,0.342182,1.342025,0.026445,38.5,214,0,0,0,0,0,1,0
136427,136427,0.986567,0.692284,-0.528565,46.3,42,0,0,0,0,0,1,0


In [16]:
# 4. Numeric type - 'Torque [Nm]'
# : RobustScaling
scaler = RobustScaler()
X_train.loc[: ,'Torque [Nm]'] = scaler.fit_transform(X_train[['Torque [Nm]']])
X_test.loc[:,'Torque [Nm]'] =  scaler.transform(X_test[['Torque [Nm]']])
X_test

/tmp/ipykernel_20/1995859707.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.loc[: ,'Torque [Nm]'] = scaler.fit_transform(X_train[['Torque [Nm]']])
/tmp/ipykernel_20/1995859707.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.loc[:,'Torque [Nm]'] =  scaler.transform(X_test[['Torque [Nm]']])


,id,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],TWF,HDF,PWF,OSF,RNF,Type_L,Type_M
0,136429,1.308759,1.125444,-0.153753,-0.208696,60,0,0,0,0,0,1,0
1,136430,0.986567,0.764477,1.388744,-1.008696,17,0,0,0,0,0,1,0
2,136431,0.771772,0.331317,0.033653,-0.234783,96,0,0,0,0,0,1,0
3,136432,0.127386,-0.246230,-0.297912,0.626087,5,0,0,0,0,0,0,1
4,136433,1.899446,1.702992,-0.038426,0.078261,114,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
90949,227378,1.308759,1.053251,-0.261872,0.000000,15,0,0,0,0,0,1,0
90950,227379,-1.053986,-0.101844,0.156188,-0.573913,31,0,0,0,0,0,1,0
90951,227380,-2.289058,-2.700806,-0.139337,0.086957,187,0,0,0,0,0,1,0
90952,227381,-0.946589,-1.545712,0.098524,-0.008696,69,0,0,0,0,0,1,0


In [17]:
# 5. Numeric type - 'Tool wear [min]'
# : MinMaxScaling
scaler = MinMaxScaler()
X_train.loc[: ,'Tool wear [min]'] = scaler.fit_transform(X_train[['Tool wear [min]']])
X_test.loc[:,'Tool wear [min]'] =  scaler.transform(X_test[['Tool wear [min]']])
X_test

/tmp/ipykernel_20/2223666855.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.loc[: ,'Tool wear [min]'] = scaler.fit_transform(X_train[['Tool wear [min]']])
/tmp/ipykernel_20/2223666855.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.loc[:,'Tool wear [min]'] =  scaler.transform(X_test[['Tool wear [min]']])


,id,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],TWF,HDF,PWF,OSF,RNF,Type_L,Type_M
0,136429,1.308759,1.125444,-0.153753,-0.208696,0.237154,0,0,0,0,0,1,0
1,136430,0.986567,0.764477,1.388744,-1.008696,0.067194,0,0,0,0,0,1,0
2,136431,0.771772,0.331317,0.033653,-0.234783,0.379447,0,0,0,0,0,1,0
3,136432,0.127386,-0.246230,-0.297912,0.626087,0.019763,0,0,0,0,0,0,1
4,136433,1.899446,1.702992,-0.038426,0.078261,0.450593,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
90949,227378,1.308759,1.053251,-0.261872,0.000000,0.059289,0,0,0,0,0,1,0
90950,227379,-1.053986,-0.101844,0.156188,-0.573913,0.122530,0,0,0,0,0,1,0
90951,227380,-2.289058,-2.700806,-0.139337,0.086957,0.739130,0,0,0,0,0,1,0
90952,227381,-0.946589,-1.545712,0.098524,-0.008696,0.272727,0,0,0,0,0,1,0


In [18]:
# 6. drop ID
id_train, id_test = X_train.pop('id'), X_test.pop('id')

## Validation set

In [19]:
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 1209)
print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape)

(109143, 12) (27286, 12) (109143,) (27286,)


## Modeling
```
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
```

In [20]:
# 1. random forest classifier

rfc = RandomForestClassifier(criterion = 'gini', random_state = 617, max_depth = 41)
rfc.fit(X_tr, y_tr)
pred_rfc = rfc.predict(X_val)

# test
roc_auc_score(y_val, pred_rfc) # default was : 0.8997767109523278

0.8803940541396528

In [21]:
# 2. gradient boosting classification

gbc = GradientBoostingClassifier(random_state = 617)
gbc.fit(X_tr, y_tr)
pred_gbc = gbc.predict(X_val)

# test
roc_auc_score(y_val, pred_gbc) # default was : 0.9010001376500757

0.875892507824726

LinearSVC_classifier = SklearnClassifier(SVC(kernel='linear',probability=True))

https://stackoverflow.com/questions/47312432/attributeerrorlinearsvc-object-has-no-attribute-predict-proba



In [22]:
# 3. stacking with SVC 1

estimators = [
    ('rf', RandomForestClassifier(random_state = 617)),
    ('gbc', GradientBoostingClassifier(random_state = 617)),
    ('abc', AdaBoostClassifier(random_state = 617))
]
clf = StackingClassifier(
    estimators = estimators, final_estimator = SVC(random_state = 617)
)

clf.fit(X_tr, y_tr).score(X_val, y_val) # 0.9957487356153338

0.9957487356153338

In [23]:
pred_clf = clf.predict(X_val)
print(roc_auc_score(y_val, pred_clf)) # 0.8749254787987183

0.8749254787987183


In [24]:
# 4. stacking with SVC 2

estimators = [
    ('svc', SVC(random_state = 617)),
    ('rf', RandomForestClassifier(random_state = 617)),
    ('abc', AdaBoostClassifier(random_state = 617))
]
clf = StackingClassifier(
    estimators = estimators, final_estimator = GradientBoostingClassifier(random_state = 617)
)

clf.fit(X_tr, y_tr).score(X_val, y_val) # 0.9953455984754086

0.9953455984754086

In [25]:
pred_clf = clf.predict(X_val)
print(roc_auc_score(y_val, pred_clf)) # 0.8747205454951934

0.8747205454951934


In [26]:
# 5. stacking with SVC 3

estimators = [
    ('svc', SVC(random_state = 617)),
    ('abc', AdaBoostClassifier(random_state = 617)),
    ('gbc', GradientBoostingClassifier(random_state = 617))
]
clf = StackingClassifier(
    estimators = estimators, final_estimator = RandomForestClassifier(random_state = 617)
)

clf.fit(X_tr, y_tr).score(X_val, y_val) # 0.9957487356153338, running time about 12min

0.9957487356153338

In [27]:
pred_clf = clf.predict(X_val)
print(roc_auc_score(y_val, pred_clf)) # 0.8771203610552201

0.8771203610552201


## Select the model

In [28]:
import time
start = time.time()

# 3rd stacking model
estimators = [
    ('svc', SVC(random_state = 617)),
    ('abc', AdaBoostClassifier(random_state = 617)),
    ('gbc', GradientBoostingClassifier(random_state = 617))
]
clf = StackingClassifier(
    estimators = estimators, final_estimator = RandomForestClassifier(random_state = 617)
)

clf.fit(X_train, y_train)

print(f"{time.time()-start:.4f} sec")

pred = clf.predict_proba(X_test)

893.3529 sec


## Submit

In [29]:
submission = pd.DataFrame({'id':id_test, 'Machine failure':pred[:, 1]})
submission

,id,Machine failure
0,136429,0.0
1,136430,0.0
2,136431,0.0
3,136432,0.0
4,136433,0.0
...,...,...
90949,227378,0.0
90950,227379,0.0
90951,227380,0.0
90952,227381,0.0


In [30]:
submission.to_csv('submission.csv', index = False)